In [ ]:
import sympy
from sympy import lambdify
import dill
dill.settings["recurse"] = True
from sympy import Rational as R
from tqdm.notebook import tqdm

In [ ]:
reference_x, reference_y = sympy.symbols("x y")
reference_x_1, reference_x_2, reference_x_3 = sympy.symbols("x_1:4")
reference_y_1, reference_y_2, reference_y_3 = sympy.symbols("y_1:4")
f_interpolation = sympy.Matrix(sympy.symbols("f_1:4"))

In [ ]:
monomial_basis = sympy.Matrix([
    1,
    reference_x,
    reference_y,
])

V = sympy.zeros(3, 3)

for dofidx, basis in enumerate(monomial_basis):
    
    V[dofidx, 0] = basis.subs({reference_x: R(1,2), reference_y: R(1,2)})
    V[dofidx, 1] = basis.subs({reference_x: 0, reference_y: R(1,2)})
    V[dofidx, 2] = basis.subs({reference_x: R(1,2), reference_y: 0})
    
crouzeix_raviart_basis = V.inv() @ monomial_basis

In [ ]:
global_x = crouzeix_raviart_basis[0]*reference_x_1 + crouzeix_raviart_basis[1]*reference_x_2 + crouzeix_raviart_basis[2]*reference_x_3
global_y = crouzeix_raviart_basis[0]*reference_y_1 + crouzeix_raviart_basis[1]*reference_y_2 + crouzeix_raviart_basis[2]*reference_y_3

J = sympy.Matrix([global_x, global_y]).jacobian([reference_x, reference_y]).inv()

In [ ]:
u = sympy.Matrix([sympy.Function("u")(reference_x, reference_y)])
v = sympy.Matrix([sympy.Function("v")(reference_x, reference_y)])

In [ ]:
grad_u = u.jacobian([reference_x, reference_y])@J
grad_v = v.jacobian([reference_x, reference_y])@J

weak_form = grad_u@grad_v.T

In [ ]:
N = len(crouzeix_raviart_basis)

weak_form_functional = [[0 for i in range(N)] for j in range(N)]
weak_form_right_part = [[0 for i in range(N)] for j in range(N)]

for idx, jdx in tqdm([(idx, jdx) for idx in range(N) for jdx in range(N)]):
    first = crouzeix_raviart_basis[idx]
    second = crouzeix_raviart_basis[jdx]

    A = weak_form.subs({u[0]:first, v[0]:second})
    A = sympy.simplify(A)
    B = first * second
    B = sympy.simplify(B)

    if not A.has(reference_x, reference_y):
        sym_int_A = R(1, 2) * A
    else:
        sym_int_A = sympy.integrate(A, (reference_y, 0, R(1) - reference_x), (reference_x, 0, R(1)))
    sym_int_B = sympy.integrate(B, (reference_y, 0, R(1) - reference_x), (reference_x, 0, R(1)))

    weak_form_functional[idx][jdx] = sym_int_A
    weak_form_right_part[idx][jdx] = sym_int_B

weak_form_functional = sympy.Matrix(weak_form_functional)
weak_form_right_part = sympy.Matrix(weak_form_right_part)

weak_form_functional = weak_form_functional * abs(J.inv().det())
weak_form_right_part = weak_form_right_part * abs(J.inv().det())

In [ ]:
weak_form_functional = sympy.Matrix(sympy.simplify(weak_form_functional))
weak_form_right_part = sympy.Matrix(sympy.simplify(weak_form_right_part))
f_interpolation = sympy.Matrix(f_interpolation)

In [ ]:
lambdify_symbols = [reference_x_1, reference_y_1, reference_x_2, reference_y_2, reference_x_3, reference_y_3]

In [ ]:
weak_form_functional_lambdified = sympy.lambdify(lambdify_symbols, weak_form_functional, cse=True)

In [ ]:
weak_form_right_part_lambdified = sympy.lambdify([*lambdify_symbols, *f_interpolation], weak_form_right_part@f_interpolation, cse=True)

In [ ]:
interpolation_function = sympy.lambdify([*lambdify_symbols, *f_interpolation, reference_x, reference_y], (crouzeix_raviart_basis.T@f_interpolation)[0, 0], cse=True)

In [ ]:
mapping_function = sympy.Matrix([global_x, global_y])
mapping_function_lambdified = sympy.lambdify([*lambdify_symbols, reference_x, reference_y], mapping_function[:, 0], cse=True)

In [ ]:
dill.dump(weak_form_functional_lambdified, open("../calculations/nonconforming_linear_crouzeix_raviart_weak_form_functional", "wb"))
dill.dump(weak_form_right_part_lambdified, open("../calculations/nonconforming_linear_crouzeix_raviart_weak_form_right_part", "wb"))
dill.dump(interpolation_function, open("../calculations/nonconforming_linear_crouzeix_raviart_basis", "wb"))
dill.dump(mapping_function_lambdified, open("../calculations/nonconforming_linear_crouzeix_raviart_mapping_function", "wb"))